In [1]:
!rm -r .ipynb_checkpoints
from feast.repo_operations import apply_total
from feast.repo_config import load_repo_config
from pathlib import Path

repo = Path('/home/jovyan/feast-spark/feature_repo/')

repo_config = load_repo_config(repo)
apply_total(repo_config, repo, True)

rm: cannot remove '.ipynb_checkpoints': No such file or directory
Created entity entity_id
Created feature view my_statistics

Created sqlite table repo_my_statistics



/opt/conda/lib/python3.9/site-packages/feast/feature_view.py:100: DeprecationWarning: The argument 'input' is being deprecated. Please use 'batch_source' instead. Feast 0.13 and onwards will not support the argument 'input'.
  warnings.warn(


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timezone
from sklearn.datasets import make_hastie_10_2
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

def generate_entities(size):
    return np.random.choice(size, size=size, replace=False)

entities=generate_entities(101)

entity_df = pd.DataFrame(data=entities, columns=['entity_id'])
entity_df["event_timestamp"]=datetime(2021, 1, 3, 23, 59, 42, tzinfo=timezone.utc)
entity_df = entity_df[entity_df.entity_id==100]

entity_df

,entity_id,event_timestamp
55,100,2021-01-03 23:59:42+00:00


In [4]:
from feast import FeatureStore
import pandas as pd

import time
from feast_custom_offline_store.spark import SparkOfflineStore

SparkOfflineStore.spark = spark
store = FeatureStore(repo_path=".")

start_time = time.time()
training_df = store.get_historical_features(
    entity_df=entity_df, 
    features = [
        'my_statistics:f0',
        'my_statistics:f1',
        'my_statistics:f2',
        'my_statistics:f3',
        'my_statistics:f4',
        'my_statistics:f5',
        'my_statistics:f6',
        'my_statistics:f7',
        'my_statistics:f8',
        'my_statistics:f9',
        'my_statistics:y',
    ],
).to_df()


print("--- %s seconds ---" % (time.time() - start_time))

training_df

--- 2.3760743141174316 seconds ---


,entity_id,event_timestamp,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,y
0,100,2021-01-03 23:59:42,0.868752,0.029004,-1.086818,1.968534,0.374946,-1.285755,-0.280189,1.455902,0.148187,0.300593,1.0
